# Práctica FIFA

### Joaquín González Alcover

El objetivo de esta práctica es intentar predecir el precio de los jugadores sólo analizando los datos de los jugadores.



## Importar librerías

Lo primero de todo, tenemos que importar las librerías que utilizaremos.

In [ ]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

## Leer los datos

Para cargar los datos del csv utilizaremos la librería pandas.

In [ ]:
df = pd.read_csv(os.path.join("../in/fifa.csv"))

Comprobamos que ha leido correctamente el fichero.

In [ ]:
df.head()

## Análisis de los datos

Lo primero de todo, para poder ver todos los datos vamos a quitar el máximo de columnas que se pueden mostrar.

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.describe()

Como podemos observar, las dos primeras columnas, correspondientes a los identificadores de cada jugador, no aportan datos relevantes, por lo que las quitamos.

In [ ]:
df = df.iloc[:, 2:]
df

Tanto la foto del jugador, como la de la bandera y la de su club, no aportan información por lo que procedemos a quitarlas.

In [ ]:
df = df.drop(columns=['Photo', 'Flag', 'Club Logo'])
df

Lo siguiente que haremos es comprobar que columnas y filas contienen NaNs.

In [ ]:
df.columns[df.isna().any()]

In [ ]:
len(df.columns[df.isna().any()].tolist())

In [ ]:
df[df.isna().any(axis=1)]

Como podemos observar, tenemos NaNs en la mayoría de columnas y en todas las filas de nuestro conjunto de datos, por tanto no podemos deshacernos directamente de todas ellas.

Comprobamos la columna 'Loaned From'.

In [ ]:
df = df.rename({"Loaned From": "LoanedFrom"}, axis='columns')
df.loc[df.LoanedFrom != df.LoanedFrom]

Como podemos ver, la mayor parte de los jugadores no son cedidos, por tanto podemos eliminar la columna 'LoanedFrom' que está en su mayor parte compuesta por NaNs.

In [ ]:
df = df.drop(columns='LoanedFrom')
df

El resto de columnas tienen más información por lo que no las eliminaremos.

Antes de poder intentar predecir el valor de los jugadores, deberemos hacer que la columna Value, la que vamos a tratar de predecir, contenga valores numéricos. Por lo que tendremos que tranformar los precios, los cuales están en una notación que tiene delante el símbolo '€' y detrás del número 'M' o 'K' que corresponde a millón y millar respectivamente.

In [ ]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

Ahora aplicamos la función a cada valor de la columna 'Value'.

In [ ]:
df["Value"] = df["Value"].apply(value_to_float)
df.head()

## Predicción

Finalmente vamos a intentar predecir el valor de los jugadores.

Por motivos obvios, el valor de un jugador viene dado por su rendimiento general, por ello, lo primero que haremos es ver si depende solamente de ello o no, tratando de predecir el valor por su columna 'Overall', también puede estar relacionado con el potencial del jugador, 'Potential', y su 'Special'. Estas columnas no hay que tratarlas de ninguna forma ya que como hemos visto antes, no contienen ningún valor NaN.

In [ ]:
val = df.pop("Value")

In [ ]:
to_drop = [c for c in df.columns if not c in ('Overall', 'Potential', 'Special')]

df_test = df
df_test = df_test.drop(to_drop, axis=1)
df_test

De esta forma tenemos en valores numéricos tanto el valor del jugador que queremos predecir, así como el valor de 'Overall'.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_test, val, test_size=0.33, random_state=42)

In [ ]:
len(X_train)

Ahora entrenaremos el modelo de regresión lineal.

In [ ]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [ ]:
preds = reg.predict(X_test)

In [ ]:
preds[0]

In [ ]:
y_test[0]

In [ ]:
r2_score(preds, y_test)

Como podemos observar, las predicciones no son las esperadas, lo que significa que las variables escogidas no están relacionadas con el valor del jugador o que hay variables de las que también depende dicho valor.

Probemos quitando la columna 'Special' puesto que como podemos ver en De Gea es especialmente baja a pesar de que su valor es de los más altos, de esto se puede deducir que dicho valor 'Special' no se corresponde con los jugadores que juegan en la posición de portero (GK), y podría llegar a confundir.

In [ ]:
to_drop = [c for c in df.columns if not c in ('Overall', 'Potential')]

df_test = df
df_test = df_test.drop(to_drop, axis=1)
df_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_test, val, test_size=0.33, random_state=42)

In [ ]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [ ]:
preds = reg.predict(X_test)

In [ ]:
preds[0]

In [ ]:
y_test[0]

In [ ]:
r2_score(preds, y_test)

Vemos que nuestro r2_score a aumentado, aunque sea bastante poco, esto quiere decir que vamos por buen camino y 'Special' no está directamente relacionado al valor del jugador.

r2_score anterior: -0.34993937335881875  
r2_score actual:   -0.3496533478047261

Otra variable relacionada con el valor del jugador podría ser el sueldo mensual que éste recibe, cuanto mejor sea un jugador, más cobra, o lo que es lo mismo en cualquier ámbito, el que ofrece mejores resultados obtiene una mejor recompensa. Para estudiar esta posibilidad, tenemos que transformar el contenido de la columna 'Wage' en valores numéricos igual que hicimos con el valor de los jugadores.

In [ ]:
df["Wage"] = df["Wage"].apply(value_to_float)
df.head()

In [ ]:
to_drop = [c for c in df.columns if not c in ('Overall', 'Potential', 'Wage')]

df_test = df
df_test = df_test.drop(to_drop, axis=1)
df_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_test, val, test_size=0.33, random_state=42)

In [ ]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [ ]:
preds = reg.predict(X_test)

In [ ]:
preds[0]

In [ ]:
y_test[0]

In [ ]:
r2_score(preds, y_test)

Como podemos observar, estábamos en lo cierto. Cuanto mejor es el jugador, más cobra.  

He querido comprobar que tanto 'Overall' como 'Potencial' son variables a tener en cuenta, por lo que he repetido este procedimiento con las distintas combinaciones de estas tres variables y estos son los resultados obtenidos:  

|**'Overall'** | **'Potencial** | **'Wage'** | **r2_score**|  
|:------------:|:--------------:|:----------:|:-----------:|
    Si    |     Si     |   Si   | 0.7619581928069343  
    No    |     No     |   Si   | 0.7168292353438137  
    Si    |     No     |   Si   | 0.7471879227405753  
    No    |     Si     |   Si   | 0.7555372125080189  
    
Como esperábamos, el mejor resultado lo obtenemos al realizar la predicción con las tres variables, por lo que podemos afirmar que las tres están relacionas con el valor del jugador.

Otra de las variables que podrían estar directamente relacionadas con el valor de un jugador es la cláusula de liberación, 'Release Clause', ya que los clubes pedirían una mayor compensación por perder a sus mejores jugadores mientras su contrato sigue activo. Por tanto, cuanto mejor sea el jugador, mayor es su valor y mayor debería ser su cláusula de liberación. De la misma forma que hicimos con el valor y el sueldo de los jugadores, debemos transformar el contenido de la columna 'Release Clause' en valores numéricos.

In [ ]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = str(x)
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [ ]:
df["Release Clause"] = df["Release Clause"].apply(value_to_float)
df.head()

In [ ]:
to_drop = [c for c in df.columns if not c in ('Overall', 'Potential', 'Wage', 'Release Clause')]

df_test = df
df_test = df_test.drop(to_drop, axis=1)
df_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_test, val, test_size=0.33, random_state=42)

In [ ]:
reg = linear_model.LinearRegression().fit(X_train, y_train)

In [ ]:
preds = reg.predict(X_test)

In [ ]:
preds[0]

In [ ]:
y_test[0]

In [ ]:
r2_score(preds, y_test)

Como podemos ver, el r2_score nos da aproximadamente 0.96, bastante cerca del 1, lo que significa que ha hecho un buen entrenamiento y hace muy bien las predicciones.

## Conclusión 

En conclusión, podemos afirmar que a partir de los datos de un jugador, podemos predecir su valor. Por lo que el valor de un jugador está intrínsecamente relacionado a sus características.